In [16]:
import pandas as pd
import git
import mlflow.keras
import pandas as pd
import os
import numpy as np
from gensim.models.wrappers import FastText

import tensorflow_hub as hub
from tensorflow.keras.callbacks import *
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.regularizers import *
from tensorflow.keras.utils import *
from tensorflow.keras.preprocessing import *
from tensorflow.keras import callbacks
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import *

from sklearn.metrics import *
from sklearn.model_selection import train_test_split

In [17]:
# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

In [18]:
path = '/home/joshib/cs_data/citation-bio-labelled-data-2020-06-08~15:13:57.766608/nlp/exps/output/2020-06-08~15:13:57.766608'
#data_df = pd.read_csv(path+'/data-2020-06-08~15:14:01.185827.csv')

## LSTM with random embedding model

In [32]:
%%time
#train_data_path = '/home/joshib/cs_data/citation-bio-labelled-data-2020-06-09~06:30:11.273245/nlp/exps/output/2020-06-09~06:30:11.273245'
#train_data_path = '/nlp/cs_data/2020-06-09~06:30:11.273245'
train_data_path = '/nlp/cs_data_full'

pattern = "*#*#"
sentences = []
sent_tags = []
len_arr = []
for fpath in os.listdir(train_data_path):
    if fpath not in ['data-gen-config.json', 'data_generation_stats.csv'] and ".csv" in fpath:
        fpath = os.path.join(train_data_path, fpath)
        df = pd.read_csv(fpath, index_col=0)
        df.fillna("\n", axis=1, inplace=True)
        len_arr.append(df.shape[0])
        sentences.append(pattern.join(df.x))
        sent_tags.append(pattern.join(df.y))

CPU times: user 2min 46s, sys: 1.06 s, total: 2min 47s
Wall time: 2min 47s


In [34]:
import numpy
print("Min: %d"%numpy.min(len_arr))
print("Max: %d"%numpy.max(len_arr))
print("Mean: %d"%numpy.mean(len_arr))
print("Median: %d"%numpy.median(len_arr))

Min: 26
Max: 7272
Mean: 1602
Median: 1543


In [35]:
%%time
maxlen = max(len_arr)
X = [[w for w in s.split(pattern)] for s in sentences]
y = [[p for p in t.split(pattern)] for t in sent_tags]

new_X = []
new_y = []
for ind in range(len(X)):
    new_seq = []
    new_tag = []
    for i in range(maxlen):
        try:
            new_seq.append(X[ind][i])
            new_tag.append(y[ind][i])
        except:
            new_seq.append("PADword")
            new_tag.append("I-CIT")
    new_X.append(new_seq)
    new_y.append(new_tag)

CPU times: user 2min 59s, sys: 4.77 s, total: 3min 3s
Wall time: 3min 3s


In [36]:
embed_size = 300 # how big is each word vector
max_features = 100000 # how many unique words to use (i.e num rows in embedding vector)
#maxlen = 1000 # max number of words in a question to use

In [37]:
%%time
tags = df.y.unique()
tags2index = {'B-CIT': 1, 'I-CIT': 0}
new_y_enc = np.array([[tags2index[w] for w in s] for s in new_y])
#y = pad_sequences(maxlen=maxlen, sequences=y, padding="post", value=tags2index["I-CIT"])

CPU times: user 45.4 s, sys: 1.31 s, total: 46.7 s
Wall time: 46.6 s


In [38]:
X_train_orig, X_test_orig, y_train, y_test = train_test_split(new_X, new_y_enc, test_size=0.1, random_state=42)

tokenizer = Tokenizer(num_words=max_features, oov_token=1)
tokenizer.fit_on_texts(list(X_train_orig))
X_train = np.array(tokenizer.texts_to_sequences(X_train_orig))
X_test  = np.array(tokenizer.texts_to_sequences(X_test_orig))

## Keras with pre-trained fast text embedding

In [39]:
%%time
wiki_model = FastText.load_fasttext_format('/nlp/cc.en.300.bin')

CPU times: user 3min 9s, sys: 3.94 s, total: 3min 13s
Wall time: 3min 36s


In [40]:
%%time
def load_embedding_matrix(embeddings, nb_words, word_index, embed_dim):  # load embeddings
    embeddings_index = {}
    for word in embeddings.wv.vocab:
        embeddings_index[word] = embeddings[word]
    print('Found %s word vectors.' % len(embeddings_index))

    words_not_found = []
    embedding_matrix = np.zeros((nb_words, embed_dim))
    for word, i in word_index.items():
        if i >= nb_words:
            continue
        try:
            embedding_vector = embeddings[word]
            embedding_matrix[i] = embedding_vector
        except:
            words_not_found.append(word)
    print('number of null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))
    return embedding_matrix


embedding_matrix = load_embedding_matrix(wiki_model, max_features, tokenizer.word_index, 300)

Found 2000000 word vectors.
number of null word embeddings: 10138
CPU times: user 8.06 s, sys: 230 ms, total: 8.29 s
Wall time: 8.28 s


In [41]:
#np.save('embedding_matrix.npy', embedding_matrix)
#embedding_matrix = np.load('./embedding_matrix.npy')

In [42]:
%%time
model = Sequential()
model.add(Embedding(max_features, 300, weights=[embedding_matrix], trainable=True, input_length= maxlen))
model.add(LSTM(50, return_sequences=True))
model.add(TimeDistributed(Dense(1, activation="sigmoid")))
# model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.compile(
    optimizer=Adam(
        learning_rate=0.001,
        beta_1=0.0,
        beta_2=0.0,
        epsilon=1e-05,
        amsgrad=False,
    ),
    loss='binary_crossentropy',
    metrics=['accuracy']
)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 7272, 300)         30000000  
_________________________________________________________________
lstm_1 (LSTM)                (None, 7272, 50)          70200     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 7272, 1)           51        
Total params: 30,070,251
Trainable params: 30,070,251
Non-trainable params: 0
_________________________________________________________________
CPU times: user 749 ms, sys: 374 ms, total: 1.12 s
Wall time: 493 ms


In [43]:
def calulate_ser_jer(y_true, y_pred, keep_tag):
    """
    :param y_true:
    :param y_pred:
    :param keep_tag:
    :return:
    """
    tn, fp, fn, tp = confusion_matrix(y_true=y_true, y_pred=y_pred).ravel()
    ser = 1.0
    if (tp + fp) > 0.0:
        ser = fp / float(tp + fp)
    jer = 1.0
    if (tp + fn) > 0.0:
        jer = fn / float(tp + fn)
    return ser, jer

class Metrics(callbacks.Callback):
    def __init__(self, train_data, validation_data):
        super(Metrics, self).__init__()
        self.validation_data = validation_data
        self.train_data = train_data
        
    def on_train_begin(self, logs={}):
        self.val_ser = []
        self.val_jer = []
        
    def on_epoch_end(self, epoch, logs):
#         if self.validation_data is None:
#             return 
        #print(self.validation_data)
        #print(self.model.get_weights()[0])
        vx = self.validation_data[0]
        vy = self.validation_data[1]
        vx = np.array(vx)
        pred_y = None
        pred_y = self.model.predict(vx)
        py = np.argmax(pred_y, axis=-1)
        #vy = np.argmax(vy, axis=-1)
        vy = np.array(vy).flatten()
        py = np.array(py).flatten()
        ser, jer = calulate_ser_jer(vy, py, 1)
        print(ser, jer)
        self.val_ser.append(ser)
        self.val_jer.append(jer)
        logs["val_ser"] = ser
        logs["val_jer"] = jer
        print(f"— val_ser: {ser} — val_jer: {jer}")

#         gold_x, gold_y, x = df_to_input()
#         pred_y = self.model.predict(x)
#         py = np.argmax(pred_y, axis=-1)[0]
#         gy = np.argmax(gold_y, axis=-1)
#         data = []
#         for i in range(len(gold_x)):
#             data.append({"x": gold_x[i], "gold": gy[i], "pred": py[i]})
#         d = pd.DataFrame(data)
#         d.to_csv("./" + str(epoch) + ".predictions.on.sample.csv")
        return 

In [44]:
metrics = Metrics([X_train, y_train], [X_test, y_test])

In [ ]:
%%time
#early_stop= EarlyStopping(monitor='val_loss',patience=3,verbose=0,mode='min',restore_best_weights=False, min_delta=0.0001)
history = model.fit(X_train, y_train, verbose=1, epochs=200, validation_data=(X_test, y_test), callbacks=[metrics])
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16, 6))
axes[0].plot(history.history['accuracy'])
axes[0].plot(history.history['val_accuracy'])
axes[0].title.set_text('model accuracy')
axes[0].set_ylabel('accuracy')
axes[0].set_xlabel('epoch')
axes[0].legend(['train', 'val'], loc='upper left')

axes[1].plot(history.history['loss'])
axes[1].plot(history.history['val_loss'])
axes[1].title.set_text('model loss')
axes[1].set_ylabel('loss')
axes[1].set_xlabel('epoch')
axes[1].legend(['train', 'val'], loc='upper left')
fig.tight_layout()

Epoch 1/200
1095/1095 [==============================] - ETA: 0s - loss: 0.0090 - accuracy: 0.99751.0 1.0
— val_ser: 1.0 — val_jer: 1.0
1095/1095 [==============================] - 715s 653ms/step - loss: 0.0090 - accuracy: 0.9975 - val_loss: 4.4780e-05 - val_accuracy: 1.0000 - val_ser: 1.0000 - val_jer: 1.0000
Epoch 2/200
1095/1095 [==============================] - ETA: 0s - loss: 2.3828e-05 - accuracy: 1.00001.0 1.0
— val_ser: 1.0 — val_jer: 1.0
1095/1095 [==============================] - 713s 651ms/step - loss: 2.3828e-05 - accuracy: 1.0000 - val_loss: 1.5238e-05 - val_accuracy: 1.0000 - val_ser: 1.0000 - val_jer: 1.0000
Epoch 3/200
1095/1095 [==============================] - ETA: 0s - loss: 1.0485e-05 - accuracy: 1.00001.0 1.0
— val_ser: 1.0 — val_jer: 1.0
1095/1095 [==============================] - 712s 650ms/step - loss: 1.0485e-05 - accuracy: 1.0000 - val_loss: 9.5215e-06 - val_accuracy: 1.0000 - val_ser: 1.0000 - val_jer: 1.0000
Epoch 4/200
1095/1095 [======================

In [ ]:
X_test.shape

### Keras with random embedding

In [15]:
%%time
model = Sequential()
model.add(Embedding(max_features, embed_size, input_length=maxlen))
model.add(Bidirectional(LSTM(512, return_sequences=True)))
#model.add(Bidirectional(LSTM(100, return_sequences=True)))
model.add(TimeDistributed(Dense(1, activation="sigmoid")))
# # model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.compile(
    optimizer=tf.keras.optimizers.Adam(
        learning_rate=0.005,
        beta_1=0.0,
        beta_2=0.0,
        epsilon=1e-05,
        amsgrad=False,
    ),
    loss='binary_crossentropy',
    metrics=['accuracy']
)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 2223, 300)         30000000  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 2223, 1024)        3330048   
_________________________________________________________________
time_distributed_1 (TimeDist (None, 2223, 1)           1025      
Total params: 33,331,073
Trainable params: 33,331,073
Non-trainable params: 0
_________________________________________________________________
CPU times: user 1.82 s, sys: 309 ms, total: 2.13 s
Wall time: 985 ms


In [16]:
model.fit(X_train, y_train, verbose=1, epochs=5, validation_data=(X_test, y_test))

Train on 400 samples, validate on 45 samples
Epoch 1/5
400/400 [==============================] - 21s 53ms/sample - loss: 0.2213 - accuracy: 0.9592 - val_loss: 0.1568 - val_accuracy: 0.9803
Epoch 2/5
400/400 [==============================] - 17s 43ms/sample - loss: 0.1173 - accuracy: 0.9802 - val_loss: 0.0976 - val_accuracy: 0.9803
Epoch 3/5
400/400 [==============================] - 17s 43ms/sample - loss: 0.1049 - accuracy: 0.9802 - val_loss: 0.1092 - val_accuracy: 0.9803
Epoch 4/5
400/400 [==============================] - 17s 43ms/sample - loss: 0.1015 - accuracy: 0.9802 - val_loss: 0.0960 - val_accuracy: 0.9803
Epoch 5/5
400/400 [==============================] - 17s 43ms/sample - loss: 0.0984 - accuracy: 0.9805 - val_loss: 0.1015 - val_accuracy: 0.9803


### Code check

In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
import tensorflow as tf
#physical_devices = tf.config.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

/home/joshib/.cache/pypoetry/virtualenvs/dp-dpo-citation-bio-trainer-FaGiNfZ9-py3.7/lib/python3.7/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
X = np.random.randint(1000, size=(10000, 80))
y = np.random.choice([0,1], size=10000)

In [3]:
model = Sequential()
model.add(Embedding(1000, 50, input_length = 80))
model.add(LSTM(10))
model.add(Dense(1, activation='sigmoid'))
print ('Compiling...')
model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])
print(model.summary())

Compiling...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 80, 50)            50000     
_________________________________________________________________
lstm (LSTM)                  (None, 10)                2440      
_________________________________________________________________
dense (Dense)                (None, 1)                 11        
Total params: 52,451
Trainable params: 52,451
Non-trainable params: 0
_________________________________________________________________
None


In [4]:
import tensorflow as tf
tf.__version__

'2.2.0'

In [5]:
model.fit(X,y)

UnknownError:  [_Derived_]  Fail to find the dnn implementation.
	 [[{{node CudnnRNN}}]]
	 [[sequential/lstm/StatefulPartitionedCall]]
	 [[gradient_tape/sequential/embedding/embedding_lookup/Reshape/_30]] [Op:__inference_train_function_3031]

Function call stack:
train_function -> train_function -> train_function


In [ ]:
model

In [6]:
oth_df = pd.read_csv("/nlp/other_ref_citations_from_64k_data_set.csv")

In [7]:
oth_df.head()

,Unnamed: 0,pii,text,label
0,0,S0272884218334801.csv,"V.Y. Osipov, A.M. Panich, A.V. Baranov, Commen...",/*/ce:bibliography/ce:bibliography-sec/ce:bib-...
1,1,S0144861718308208.csv,100 of the World's Worst Invasive Alien Specie...,/*/ce:bibliography/ce:bibliography-sec/ce:bib-...
2,2,S0144861718308208.csv,Plastics - the Facts 2015. (2015). http://www....,/*/ce:bibliography/ce:bibliography-sec/ce:bib-...
3,3,S0959652618324478.csv,"Adiansyah, J.S., Haque, N., Rosano, M., Biswas...",/*/ce:bibliography/ce:bibliography-sec/ce:bib-...
4,4,S0959652618324478.csv,"Ashley, S.F., Fenner, R.A., Nuttall, W.J., Par...",/*/ce:bibliography/ce:bibliography-sec/ce:bib-...


In [12]:
oth_df.text.head(50).tolist()

['V.Y. Osipov, A.M. Panich, A.V. Baranov, Comment on “Carbon structure in nanodiamonds elucidated from Raman spectroscopy” by V.I. Korepanov et al., Carbon 127 (2018) 193-194. ',
 "100 of the World's Worst Invasive Alien Species. (2013). http://www.iucngisd.org/gisd/100_worst.php. ",
 'Plastics - the Facts 2015. (2015). http://www.plasticseurope.org/Document/plastics---the-facts-2015.aspx. ',
 'Adiansyah, J.S., Haque, N., Rosano, M., Biswas, W., 2017. Application of a life cycle assessment to compare environmental performance in coal mine tailings management. J. Environ. Manage. 199, 181–191. https://doi.org/10.1016/j.jenvman.2017.05.050 ',
 'Ashley, S.F., Fenner, R.A., Nuttall, W.J., Parks, G.T., 2015. Life-cycle impacts from novel thorium-uranium-fuelled nuclear energy systems. Energy Convers. Manag. 101, 136–150. https://doi.org/10.1016/j.enconman.2015.04.041 ',
 'Awuah-Offei, K., Adekpedjou, A., 2011. Application of life cycle assessment in the mining industry. Int. J. Life Cycle A